In [1]:
import pandas as pd
from sklearn.metrics import classification_report
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline,AutoModelForSeq2SeqLM

import torch
print(torch.__version__)
print(torch.cuda.is_available())
from transformers.utils import is_torch_available, is_tf_available

print("Torch disponible:", is_torch_available())
print("TensorFlow disponible:", is_tf_available())



c:\Users\Jaime Robledo\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2.6.0.dev20241112+cu121
True
Torch disponible: True
TensorFlow disponible: False


In [4]:
df = pd.read_csv('INPUT/emociones.csv')
df = df[['opinion', 'label']]
df = df.dropna()
df = df.sample(30, random_state=42).reset_index(drop=True)
print(df)


                       opinion     label
0          Es un gran servicio  positiva
1                   Es regular   neutral
2          Es un producto útil  positiva
3                 Es un fraude  negativa
4               Me gusta mucho  positiva
5               Es un desastre  negativa
6           Es un mal servicio  negativa
7        Es un producto inútil  negativa
8           No lo recomendaría  negativa
9       Me encantó el producto  positiva
10                 No me gusta  negativa
11               Es fantástico  positiva
12                  Me encanta  positiva
13         Es un buen producto  positiva
14                Es muy bueno  positiva
15          Es un mal servicio  negativa
16    Es una pérdida de dinero  negativa
17         No es nada especial   neutral
18       Es un producto normal   neutral
19                     Te odio  negativa
20         Es un buen servicio  positiva
21         Es un gran producto  positiva
22      Es un producto decente   neutral
23  Es un produc

In [ ]:
prompt_base = f"Clasifica la siguiente opinión en base a como interpretes la actitud del mensaje, dicha actitud puede ser vista como positiva, negativa o neutral: {opinion}"

zero_shot_prompt = f"Clasifica la siguiente opinión como positiva, negativa o neutral: {opinion}"


few_shot_prompt = f"""
Aquí tienes algunas opiniones clasificadas:

Opinión: 'Me encanta este producto. Es increíble.' → Sentimiento: positivo
Opinión: 'Este servicio es terrible, nunca lo recomendaría.' → Sentimiento: negativo
Opinión: 'El artículo está bien, pero no es excepcional.' → Sentimiento: neutral

Clasifica esta opinión: {opinion}
"""

cot_prompt = f"""
Pensando en la siguiente opinión:

Opinión: '{opinion}'

El sentimiento puede ser positivo, negativo o neutral. ¿Qué opinas?
"""


prompts = {
    "prompt": prompt_base,
    "zero-shot": zero_shot_prompt,
    "few-shot": few_shot_prompt,
    "chain-of-thought": cot_prompt
}

NameError: name 'prompt_base' is not defined

In [8]:

generator = pipeline('text-generation', model="gpt2", device=0)

# Lista para almacenar las predicciones
predictions_base = []
predictions_zero_shot = []
predictions_few_shot = []
predictions_cot = []

# Clasificar cada opinión y mostrar resultado
for opinion in df['opinion']:

    # Generar predicción con Zero-shot
    prompt_zero_shot = zero_shot_prompt.format(opinion)
    result_zero_shot = generator(prompt_zero_shot, max_length=100, num_return_sequences=1)
    sentiment_zero_shot = result_zero_shot[0]["generated_text"]
    
    # Generar predicción con Few-shot
    prompt_few_shot = few_shot_prompt.format(opinion)
    result_few_shot = generator(prompt_few_shot, max_new_tokens=50, num_return_sequences=1)
    sentiment_few_shot = result_few_shot[0]["generated_text"]

    # Generar predicción con el prompt base
    prompt = prompt_base.format(opinion)
    result_base = generator(prompt, max_length=100, num_return_sequences=1)
    sentiment_base = result_base[0]["generated_text"]
    
    # Generar predicción con Chain-of-thought
    prompt_cot = cot_prompt.format(opinion)
    result_cot = generator(prompt_cot, max_length=100, num_return_sequences=1)
    sentiment_cot = result_cot[0]["generated_text"]
    
    # Asignar sentimientos de acuerdo al texto generado
    def assign_sentiment(response):
        if "positivo" in response.lower():
            return "positivo"
        elif "negativo" in response.lower():
            return "negativo"
        elif "neutral" in response.lower():
            return "neutral"
        else:
            return "desconocido"
    
    # Asignar sentimientos a cada predicción
    predictions_base.append(assign_sentiment(sentiment_base))
    predictions_zero_shot.append(assign_sentiment(sentiment_zero_shot))
    predictions_few_shot.append(assign_sentiment(sentiment_few_shot))
    predictions_cot.append(assign_sentiment(sentiment_cot))

    # Imprimir resultados
    print(f"Opinión: {opinion}\n → Zero-shot Sentimiento: {predictions_zero_shot[-1]}")
    print(f" → Few-shot Sentimiento: {predictions_few_shot[-1]}")
    print(f" → Chain-of-thought Sentimiento: {predictions_cot[-1]}")
    print(f" → Base Sentimiento: Sentimiento: {predictions_base[-1]}\n")

# Añadir las predicciones al DataFrame
df['prompt_base'] = prompt_base
df['prediccion_zero_shot'] = predictions_zero_shot
df['prediccion_few_shot'] = predictions_few_shot
df['prediccion_cot'] = predictions_cot

# Mostrar el DataFrame con los resultados
print(df[['opinion', 'label', 'prompt_base' ,'prediccion_zero_shot', 'prediccion_few_shot', 'prediccion_cot']])

Device set to use cuda:0


NameError: name 'zero_shot_prompt' is not defined

In [ ]:
classifier = pipeline('sentiment-analysis', model='nlptown/bert-base-multilingual-uncased-sentiment')

for opinion in df['opinion']:
    sentiment = classifier(opinion)
    print(f"Opinión: {opinion} → Sentimiento: {sentiment[0]['label']}")
